In [8]:
import io
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from torch.utils.tensorboard import SummaryWriter


def loadtraindata():
    path = r"./train"
    trainset = torchvision.datasets.ImageFolder(path,
                                                transform=transforms.Compose([
                                                    transforms.Resize((300, 300)),  # 将图片缩放到指定大小（h,w）
                                                    transforms.CenterCrop(300),
                                                    transforms.ToTensor()])
                                                )
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                              shuffle=True, num_workers=2)
    return trainloader

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 9)  # 卷积层
        self.pool1 = nn.MaxPool2d(2, 2)  # 池化层
        self.conv2 = nn.Conv2d(6, 10, 11)  # 卷积层
        self.conv3 = nn.Conv2d(10, 6, 9)  # 卷积层
        self.conv4 = nn.Conv2d(6, 16, 11)  # 卷积层
        self.fc1 = nn.Linear(1600, 480)  # 全连接层
        self.fc2 = nn.Linear(480, 120)
        self.fc3 = nn.Linear(120, 3)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool1(F.relu(self.conv2(x)))
        x = self.pool1(F.relu(self.conv3(x)))
        x = self.pool1(F.relu(self.conv4(x)))
        x = x.view(-1, 1600)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


classes = ('未知','女', '男')



def trainandsave():  # 训练
    writer = SummaryWriter('runs/ManOrWuman')
    trainloader = loadtraindata()
    #net = Net()
    net = reload_net()
    optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.001)
    criterion = nn.CrossEntropyLoss()
    for epoch in range(500):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = Variable(inputs), Variable(labels)

            img_grid = torchvision.utils.make_grid(inputs)
            writer.add_image('Man', img_grid)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            if i % 20 == 19:
                writer.add_scalar('training loss',running_loss / 20,epoch * len(trainloader) + i)
                # writer.add_figure('predictions vs. actuals',plot_classes_preds(net, inputs, labels),global_step=epoch * len(trainloader) + i)
                print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 20))
                running_loss = 0.0
        print(str(epoch) + '  OK!')
        torch.save(net, './out/net'+str(epoch)+'.pkl')
        torch.save(net.state_dict(), './out/net_params'+str(epoch)+'.pkl')

    writer.add_graph(Net(), inputs)
    writer.close()
    print('Finished Training')



def reload_net():
    trainednet = torch.load('./out/net23.pkl')
    return trainednet


def test_one(img_path):
    model = reload_net()
    transform_valid = transforms.Compose([
        transforms.Resize((300, 300), interpolation=4),
        transforms.ToTensor()
    ]
    )
    img = Image.open(img_path)
    img_ = transform_valid(img).unsqueeze(0)
    outputs = model(img_)
    _, indices = torch.max(outputs, 1)
    #print(indices)
    result = classes[indices]
    #print('我认为这个东西的性别是:', result)
    #print(outputs)
    return result

def main():
    trainandsave()

In [22]:
test_one(r'./train/女/tet16.jpg')

tensor([1])
我认为这个东西的性别是: 女
tensor([[-5.7683,  5.9619, -0.3153]], grad_fn=<AddmmBackward>)


In [7]:
main()

[1,    20] loss: 0.266
[1,    40] loss: 0.191
0  OK!
[2,    20] loss: 0.058
[2,    40] loss: 0.041
1  OK!
[3,    20] loss: 0.007
[3,    40] loss: 0.026
2  OK!
[4,    20] loss: 0.004
[4,    40] loss: 0.022
3  OK!
[5,    20] loss: 0.001
[5,    40] loss: 0.026
4  OK!
[6,    20] loss: 0.006
[6,    40] loss: 0.013
5  OK!
[7,    20] loss: 0.000
[7,    40] loss: 0.015
6  OK!
[8,    20] loss: 0.014
[8,    40] loss: 0.009
7  OK!
[9,    20] loss: 0.001
[9,    40] loss: 0.012
8  OK!
[10,    20] loss: 0.020
[10,    40] loss: 0.005
9  OK!
[11,    20] loss: 0.000
[11,    40] loss: 0.012
10  OK!
[12,    20] loss: 0.000
[12,    40] loss: 0.008
11  OK!
[13,    20] loss: 0.000
[13,    40] loss: 0.006
12  OK!
[14,    20] loss: 0.009
[14,    40] loss: 0.001
13  OK!
[15,    20] loss: 0.000
[15,    40] loss: 0.004
14  OK!
[16,    20] loss: 0.001
[16,    40] loss: 0.000
15  OK!
[17,    20] loss: 0.000
[17,    40] loss: 0.001
16  OK!
[18,    20] loss: 0.000
[18,    40] loss: 0.001
17  OK!
[19,    20] loss: 0.

KeyboardInterrupt: 

In [33]:
import os
os.listdir("train\女")
for name in os.listdir("train\女"):
    datali = "train\女/"
    test_one(datali + name)

tensor([1])
我认为这个东西的性别是: 女
tensor([[-5.0814,  5.2480, -0.5318]], grad_fn=<AddmmBackward>)
tensor([1])
我认为这个东西的性别是: 女
tensor([[-5.5593,  7.1158, -1.5500]], grad_fn=<AddmmBackward>)
tensor([1])
我认为这个东西的性别是: 女
tensor([[-7.4239,  6.5711,  1.1758]], grad_fn=<AddmmBackward>)
tensor([1])
我认为这个东西的性别是: 女
tensor([[-1.6561,  1.3235,  0.3479]], grad_fn=<AddmmBackward>)
tensor([1])
我认为这个东西的性别是: 女
tensor([[-2.3210,  1.7635,  0.5538]], grad_fn=<AddmmBackward>)
tensor([1])
我认为这个东西的性别是: 女
tensor([[-2.7614,  1.6235,  0.8059]], grad_fn=<AddmmBackward>)
tensor([1])
我认为这个东西的性别是: 女
tensor([[-4.8470,  3.5484,  1.6563]], grad_fn=<AddmmBackward>)
tensor([1])
我认为这个东西的性别是: 女
tensor([[-7.1622,  7.7006, -0.0117]], grad_fn=<AddmmBackward>)
tensor([1])
我认为这个东西的性别是: 女
tensor([[-4.3697,  4.8423, -0.5957]], grad_fn=<AddmmBackward>)
tensor([1])
我认为这个东西的性别是: 女
tensor([[-4.4308,  4.7253, -0.7929]], grad_fn=<AddmmBackward>)
tensor([1])
我认为这个东西的性别是: 女
tensor([[-5.4868,  7.0378, -1.6435]], grad_fn=<AddmmBackward>)
tensor([1]

In [9]:
import cv2 as cv
import os

cap = cv.VideoCapture(0)
while True:
    ret,frame = cap.read()
    cv.imwrite('RAM.jpg',frame)
    if test_one('RAM.jpg') == '女':
        print('女')
    cv.imshow('win',frame)
    os.remove('RAM.jpg')
    os.system('cls')
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv.destroyAllWindows()

女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
女
